In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url_actual_movies = "https://www.cinema-senechal.com/#actuellement"

html_actual_movies = requests.get(url_actual_movies)
soup_actual_movies = BeautifulSoup(html_actual_movies.text, 'html.parser')

data_actual_movies = []

for film in soup_actual_movies.find_all('a', class_="undefined css-erexzk"):

  lien = "https://www.cinema-senechal.com" + film['href']

# page du film pour extraire le titre
  page_film = requests.get(lien)
  soup_film = BeautifulSoup(page_film.text, 'html.parser')

  titre_film = soup_film.find('h1', class_="css-1atox0n")
  if titre_film:
    nom_film = titre_film.get_text(strip=True)
  else:
    "Aucun titre trouvé"

  # extraire le résumé du film
  overview = "Résumé inconnu"
  overview_bloc = soup_film.find('div', class_="css-9q7hex")
  if overview_bloc:
    overview_span = overview_bloc.find('span', class_="css-skxv7x")
    if overview_span:
      overview = overview_span.get_text(strip=True)
    else:
      overview = "Résumé non trouvé"
  else:
    "Résumé inconnu"
  
  # extraire la durée du film
  span_bloc = soup_film.find_all("div", class_="css-uyt4dk")
  if span_bloc and len(span_bloc) == 3:
    bloc_duree = span_bloc[0]
  elif span_bloc and len(span_bloc) > 3:
    bloc_duree = span_bloc[2]
  duree_span = bloc_duree.find('span')
  if duree_span:
    duree = duree_span.get_text(strip=True)
  else:
    duree = "Durée inconnue"

  # extraire la date de sortie du film
  if span_bloc and len(span_bloc) == 3:
    bloc_date = span_bloc[2]
  elif span_bloc and len(span_bloc) > 3:
    bloc_duree = span_bloc[4]
  date_span = bloc_date.find('span')
  if date_span:
    date_sortie = date_span.text
  else:
    date_sortie = "Date inconnue"

# extraire genre, réalisation, casting
  infosup_bloc = soup_film.find_all("strong", class_="css-y5qmm9")

  genre = "Genre inconnu"
  realisateur = "Réalisateur inconnu"
  casting = "Casting inconnu"

  if infosup_bloc and len(infosup_bloc) == 3:
      try:
        genre = infosup_bloc[0].next_sibling.strip()
        realisateur = infosup_bloc[1].next_sibling.strip()
        casting = infosup_bloc[2].next_sibling.strip()

      except IndexError:
          pass
  
  # Extraire l'affiche du film
  affiche_bloc = soup_film.find('meta', property="og:image")  # Remplacez "css-xyz-class" par la classe exacte
  if affiche_bloc:
      url_affiche = affiche_bloc['content']
  else:
      affiche_bloc = soup_film.find('meta', name="image")
      if affiche_bloc:
          url_affiche = affiche_bloc['content']
      else:
          url_affiche = "Affiche non trouvée"
  #print(soup_film.prettify()) # permet de vérifier si l'attribut src se trouve dans la balise image

  data_actual_movies.append({'Nom': nom_film, 'Lien': lien, 'Résumé': overview, 'Durée': duree, 'Date_Sortie': date_sortie,
                             'Genre': genre, 'Réalisateur': realisateur, 'Casting': casting, 'Poster': url_affiche})

df_actual_movies = pd.DataFrame(data_actual_movies)
df_actual_movies


,Nom,Lien,Résumé,Durée,Date_Sortie,Genre,Réalisateur,Casting,Poster
0,Six jours,https://www.cinema-senechal.com/films/321604-s...,"Nord de la France, 2005 : Malik, inspecteur de...",1h 41min,01/01/2025,Thriller,Juan Carlos Medina,"Sami Bouajila, Julie Gayet, Philippe Resimont",https://all.web.img.acsta.net/img/ca/57/ca5791...
1,Un ours dans le jura,https://www.cinema-senechal.com/films/323570-u...,"Michel et Cathy, un couple usé par le temps et...",1h 53min,01/01/2025,"Comédie, Thriller",Franck Dubosc,"Franck Dubosc, Laure Calamy, Benoît Poelvoorde",https://all.web.img.acsta.net/img/17/5f/175fbc...
2,Sonic 3 - le film,https://www.cinema-senechal.com/films/300949-s...,"Sonic, Knuckles et Tails se retrouvent face à ...",1h 49min,25/12/2024,"Animation, Aventure",Jeff Fowler,"Jim Carrey, Tika Sumpter, James Marsden",https://all.web.img.acsta.net/img/85/e7/85e77c...
3,Kraven The Hunter,https://www.cinema-senechal.com/films/266665-k...,KRAVEN THE HUNTER raconte la genèse sanglante ...,2h 7min,25/12/2024,"Action, Fantastique",J.C. Chandor,"Aaron Taylor-Johnson, Russell Crowe, Ariana De...",https://all.web.img.acsta.net/img/bf/75/bf759d...
4,Mufasa : Le Roi Lion,https://www.cinema-senechal.com/films/286777-m...,Rafiki raconte à la jeune lionne Kiara - la fi...,1h 58min,18/12/2024,"Animation, Concert, Aventure, Famille",Barry Jenkins,"Aaron Pierre, Kelvin Harrison Jr., Seth Rogen",https://all.web.img.acsta.net/img/f5/f2/f5f244...
5,Sous écrous,https://www.cinema-senechal.com/films/10000010...,Sammy est un étudiant en droit qui jongle entr...,1h 51min,18/12/2024,"Comédie, Action",Hakim Boughéraba,"Ichem Bougheraba, Arriles Amrani, Bernard Farcy",https://all.web.img.acsta.net/img/3e/dc/3edcfd...
6,Un Noël en famille,https://www.cinema-senechal.com/films/316394-u...,"C’est Noël ! Carole, Maire d’une petite ville,...",1h 30min,18/12/2024,Comédie,Jeanne Gottesdiener,"Noémie Lvovsky, Didier Bourdon, Alice Daubelcour",https://all.web.img.acsta.net/img/3b/07/3b0792...
7,Vingt dieux,https://www.cinema-senechal.com/films/311910-v...,"Totone, 18 ans, passe le plus clair de son te...",1h 30min,11/12/2024,Drame,Louise Courvoisier,"Clément Faveau, Maïwène Barthelemy, Luna Garret",https://all.web.img.acsta.net/img/7e/0c/7e0c02...
8,100 000 000 000 000 - cent mille milliards,https://www.cinema-senechal.com/films/310626-1...,Ils restèrent toute la nuit à discuter dans la...,1h 17min,04/12/2024,Drame,Virgil Vernier,"Zakaria Bouti, Mina Gajovic, Victoire Song",https://all.web.img.acsta.net/img/ca/26/ca2671...
9,Conclave,https://www.cinema-senechal.com/films/304895-c...,Quand le pape décède de façon inattendue et my...,2h 0min,04/12/2024,Thriller,Edward Berger,"Ralph Fiennes, Stanley Tucci, Isabella Rossellini",https://all.web.img.acsta.net/img/49/3b/493b67...


In [3]:
df_actual_movies['Date_Sortie'] = pd.to_datetime(df_actual_movies['Date_Sortie'], format='%d/%m/%Y')

In [4]:
df_actual_movies['Date_Sortie'].dtype

dtype('<M8[ns]')

In [5]:
df_actual_movies['Year'] = df_actual_movies['Date_Sortie'].dt.year


In [7]:
debut_ligne_a_supp = 21
fin_ligne_a_supp = 29

lignes_a_supp = list(range(debut_ligne_a_supp, fin_ligne_a_supp+1))

df_sans_event = df_actual_movies.drop(lignes_a_supp, axis=0)
df_sans_event

,Nom,Lien,Résumé,Durée,Date_Sortie,Genre,Réalisateur,Casting,Poster,Year
0,Six jours,https://www.cinema-senechal.com/films/321604-s...,"Nord de la France, 2005 : Malik, inspecteur de...",1h 41min,2025-01-01,Thriller,Juan Carlos Medina,"Sami Bouajila, Julie Gayet, Philippe Resimont",https://all.web.img.acsta.net/img/ca/57/ca5791...,2025
1,Un ours dans le jura,https://www.cinema-senechal.com/films/323570-u...,"Michel et Cathy, un couple usé par le temps et...",1h 53min,2025-01-01,"Comédie, Thriller",Franck Dubosc,"Franck Dubosc, Laure Calamy, Benoît Poelvoorde",https://all.web.img.acsta.net/img/17/5f/175fbc...,2025
2,Sonic 3 - le film,https://www.cinema-senechal.com/films/300949-s...,"Sonic, Knuckles et Tails se retrouvent face à ...",1h 49min,2024-12-25,"Animation, Aventure",Jeff Fowler,"Jim Carrey, Tika Sumpter, James Marsden",https://all.web.img.acsta.net/img/85/e7/85e77c...,2024
3,Kraven The Hunter,https://www.cinema-senechal.com/films/266665-k...,KRAVEN THE HUNTER raconte la genèse sanglante ...,2h 7min,2024-12-25,"Action, Fantastique",J.C. Chandor,"Aaron Taylor-Johnson, Russell Crowe, Ariana De...",https://all.web.img.acsta.net/img/bf/75/bf759d...,2024
4,Mufasa : Le Roi Lion,https://www.cinema-senechal.com/films/286777-m...,Rafiki raconte à la jeune lionne Kiara - la fi...,1h 58min,2024-12-18,"Animation, Concert, Aventure, Famille",Barry Jenkins,"Aaron Pierre, Kelvin Harrison Jr., Seth Rogen",https://all.web.img.acsta.net/img/f5/f2/f5f244...,2024
5,Sous écrous,https://www.cinema-senechal.com/films/10000010...,Sammy est un étudiant en droit qui jongle entr...,1h 51min,2024-12-18,"Comédie, Action",Hakim Boughéraba,"Ichem Bougheraba, Arriles Amrani, Bernard Farcy",https://all.web.img.acsta.net/img/3e/dc/3edcfd...,2024
6,Un Noël en famille,https://www.cinema-senechal.com/films/316394-u...,"C’est Noël ! Carole, Maire d’une petite ville,...",1h 30min,2024-12-18,Comédie,Jeanne Gottesdiener,"Noémie Lvovsky, Didier Bourdon, Alice Daubelcour",https://all.web.img.acsta.net/img/3b/07/3b0792...,2024
7,Vingt dieux,https://www.cinema-senechal.com/films/311910-v...,"Totone, 18 ans, passe le plus clair de son te...",1h 30min,2024-12-11,Drame,Louise Courvoisier,"Clément Faveau, Maïwène Barthelemy, Luna Garret",https://all.web.img.acsta.net/img/7e/0c/7e0c02...,2024
8,100 000 000 000 000 - cent mille milliards,https://www.cinema-senechal.com/films/310626-1...,Ils restèrent toute la nuit à discuter dans la...,1h 17min,2024-12-04,Drame,Virgil Vernier,"Zakaria Bouti, Mina Gajovic, Victoire Song",https://all.web.img.acsta.net/img/ca/26/ca2671...,2024
9,Conclave,https://www.cinema-senechal.com/films/304895-c...,Quand le pape décède de façon inattendue et my...,2h 0min,2024-12-04,Thriller,Edward Berger,"Ralph Fiennes, Stanley Tucci, Isabella Rossellini",https://all.web.img.acsta.net/img/49/3b/493b67...,2024


In [8]:
def minutes(heure):

	heure = heure.split(' ')
	hour = int(heure[0].replace('h', ''))
	float_part = hour
	hour_foat = float(hour - float_part)
	minutes = int(heure[1].replace('min', ''))
	h_minutes = (hour * 60) + minutes
	return h_minutes

df_sans_event['Durée_minutes'] = df_sans_event['Durée'].apply(minutes)
df_sans_event

,Nom,Lien,Résumé,Durée,Date_Sortie,Genre,Réalisateur,Casting,Poster,Year,Durée_minutes
0,Six jours,https://www.cinema-senechal.com/films/321604-s...,"Nord de la France, 2005 : Malik, inspecteur de...",1h 41min,2025-01-01,Thriller,Juan Carlos Medina,"Sami Bouajila, Julie Gayet, Philippe Resimont",https://all.web.img.acsta.net/img/ca/57/ca5791...,2025,101
1,Un ours dans le jura,https://www.cinema-senechal.com/films/323570-u...,"Michel et Cathy, un couple usé par le temps et...",1h 53min,2025-01-01,"Comédie, Thriller",Franck Dubosc,"Franck Dubosc, Laure Calamy, Benoît Poelvoorde",https://all.web.img.acsta.net/img/17/5f/175fbc...,2025,113
2,Sonic 3 - le film,https://www.cinema-senechal.com/films/300949-s...,"Sonic, Knuckles et Tails se retrouvent face à ...",1h 49min,2024-12-25,"Animation, Aventure",Jeff Fowler,"Jim Carrey, Tika Sumpter, James Marsden",https://all.web.img.acsta.net/img/85/e7/85e77c...,2024,109
3,Kraven The Hunter,https://www.cinema-senechal.com/films/266665-k...,KRAVEN THE HUNTER raconte la genèse sanglante ...,2h 7min,2024-12-25,"Action, Fantastique",J.C. Chandor,"Aaron Taylor-Johnson, Russell Crowe, Ariana De...",https://all.web.img.acsta.net/img/bf/75/bf759d...,2024,127
4,Mufasa : Le Roi Lion,https://www.cinema-senechal.com/films/286777-m...,Rafiki raconte à la jeune lionne Kiara - la fi...,1h 58min,2024-12-18,"Animation, Concert, Aventure, Famille",Barry Jenkins,"Aaron Pierre, Kelvin Harrison Jr., Seth Rogen",https://all.web.img.acsta.net/img/f5/f2/f5f244...,2024,118
5,Sous écrous,https://www.cinema-senechal.com/films/10000010...,Sammy est un étudiant en droit qui jongle entr...,1h 51min,2024-12-18,"Comédie, Action",Hakim Boughéraba,"Ichem Bougheraba, Arriles Amrani, Bernard Farcy",https://all.web.img.acsta.net/img/3e/dc/3edcfd...,2024,111
6,Un Noël en famille,https://www.cinema-senechal.com/films/316394-u...,"C’est Noël ! Carole, Maire d’une petite ville,...",1h 30min,2024-12-18,Comédie,Jeanne Gottesdiener,"Noémie Lvovsky, Didier Bourdon, Alice Daubelcour",https://all.web.img.acsta.net/img/3b/07/3b0792...,2024,90
7,Vingt dieux,https://www.cinema-senechal.com/films/311910-v...,"Totone, 18 ans, passe le plus clair de son te...",1h 30min,2024-12-11,Drame,Louise Courvoisier,"Clément Faveau, Maïwène Barthelemy, Luna Garret",https://all.web.img.acsta.net/img/7e/0c/7e0c02...,2024,90
8,100 000 000 000 000 - cent mille milliards,https://www.cinema-senechal.com/films/310626-1...,Ils restèrent toute la nuit à discuter dans la...,1h 17min,2024-12-04,Drame,Virgil Vernier,"Zakaria Bouti, Mina Gajovic, Victoire Song",https://all.web.img.acsta.net/img/ca/26/ca2671...,2024,77
9,Conclave,https://www.cinema-senechal.com/films/304895-c...,Quand le pape décède de façon inattendue et my...,2h 0min,2024-12-04,Thriller,Edward Berger,"Ralph Fiennes, Stanley Tucci, Isabella Rossellini",https://all.web.img.acsta.net/img/49/3b/493b67...,2024,120


In [14]:
df_sans_event['Genre_split'] = df_sans_event['Genre'].apply(lambda x: str(x.split(',')))

In [15]:
df_sans_event['Genre_split']

0                                          ['Thriller']
1                              ['Comédie', ' Thriller']
2                            ['Animation', ' Aventure']
3                            ['Action', ' Fantastique']
4     ['Animation', ' Concert', ' Aventure', ' Famil...
5                                ['Comédie', ' Action']
6                                           ['Comédie']
7                                             ['Drame']
8                                             ['Drame']
9                                          ['Thriller']
10         ['Comédie dramatique', ' Comédie', ' Drame']
11               ['Animation', ' Aventure', ' Famille']
12                              ['Action', ' Thriller']
13                 ['Fantastique', ' Comédie musicale']
14                                            ['Drame']
15                 ['Action', ' Thriller', ' Policier']
16         ['Comédie dramatique', ' Drame', ' Comédie']
17                                ['Épouvante-ho

In [21]:
# dico = {"Drame": "Drama", "Comédie": "Comedy", "Aventure": "Adventure", "Fantastique": "Fantastic", "Famille": "Family", "Drame": "Drama", "Comédie musicale": "Comedy-Music", 
#          "Comédie dramatique": "Comedy-Drama", "Policier": "Crime"}

# def remplacer_valeur(x):
#     trad = x
#     for key, value in dico.items():
#         if key == x:
#             trad = value
#             break
#     return trad

dico = {"Drame": "Drama", "Comédie": "Comedy", "Aventure": "Adventure", "Fantastique": "Fantastic", "Famille": "Family", "Drame": "Drama", "Comédie musicale": "Comedy-Music", 
        "Comédie dramatique": "Comedy-Drama", "Policier": "Crime"}
for key, value in dico.items():
    df_sans_event['Genre'] = df_sans_event['Genre'].apply(lambda x: x.replace(key, value))


In [23]:
df_sans_event['Genre']

0                                  Thriller
1                          Comedy, Thriller
2                      Animation, Adventure
3                         Action, Fantastic
4     Animation, Concert, Adventure, Family
5                            Comedy, Action
6                                    Comedy
7                                     Drama
8                                     Drama
9                                  Thriller
10         Comedy dramatique, Comedy, Drama
11             Animation, Adventure, Family
12                         Action, Thriller
13               Fantastic, Comedy musicale
14                                    Drama
15                  Action, Thriller, Crime
16         Comedy dramatique, Drama, Comedy
17                        Épouvante-horreur
18                           Drama, Romance
19                                   Biopic
20                                    Drama
Name: Genre, dtype: object

In [24]:
df_sans_event['Genre_split'] = df_sans_event['Genre'].apply(lambda x: str(x.split(',')))
df_sans_event['Genre_split']

0                                          ['Thriller']
1                               ['Comedy', ' Thriller']
2                           ['Animation', ' Adventure']
3                              ['Action', ' Fantastic']
4     ['Animation', ' Concert', ' Adventure', ' Fami...
5                                 ['Comedy', ' Action']
6                                            ['Comedy']
7                                             ['Drama']
8                                             ['Drama']
9                                          ['Thriller']
10           ['Comedy dramatique', ' Comedy', ' Drama']
11               ['Animation', ' Adventure', ' Family']
12                              ['Action', ' Thriller']
13                    ['Fantastic', ' Comedy musicale']
14                                            ['Drama']
15                    ['Action', ' Thriller', ' Crime']
16           ['Comedy dramatique', ' Drama', ' Comedy']
17                                ['Épouvante-ho

In [26]:
df_sans_event.to_csv('df_movies_cinema.csv')